# 1. Reading and Analysing

In [9]:
import pyodbc
import pandas as pd

server = 'DELL\\SQLEXPRESS'
database = 'NEHADB'

connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
conn = pyodbc.connect(connection_string)

query = "SELECT * FROM dbo.A2_APX"
cursor = conn.cursor()
cursor.execute(query)

rows = cursor.fetchall()
columns = [column[0] for column in cursor.description]

df = pd.DataFrame.from_records(rows, columns=columns)

print(df.head())

# Analyze the data
print("Data Overview:\n", df.head())
print("\nStatistics:\n", df.describe(include='all'))


     EXT_SEC_ID SEC_NAME SEC_TYP_CD CUSIP ISSUER_CD ISSUE_STATE_CD  \
0  abcaa1202121     None       None  None      None           None   
1  abcaa2012121     None       None  None      None           None   
2  abcaa240615n     None       None  None      None           None   
3  abcaa240615o     None       None  None      None           None   
4  abcaa240815a     None       None  None      None           None   

  ISSUE_CNTRY_CD ISSUE_DATE ULTMT_ISSUER_CD LIST_EXCH_CD  ...  \
0           None       None            None         None  ...   
1           None       None            None         None  ...   
2           None       None            None         None  ...   
3           None       None            None         None  ...   
4           None       None            None         None  ...   

  CREATE_SERVICE_DETAIL CREATE_SERVICE_CD CREATE_DATE SERVICE_ID SEC_ALT_NAME  \
0                  None              None        None       None         None   
1                  None   

# 2. Datatype Suggestion

In [2]:
dtype_mapping = {
    'int64': 'INT',
    'float64': 'FLOAT',
    'object': 'VARCHAR(MAX)', 
    'datetime64[ns]': 'DATETIME',
    'bool': 'BIT'
}

suggested_dtypes = {col: dtype_mapping[str(df[col].dtype)] for col in df.columns}

print("Suggested Data Types for SQL Server:")
print(suggested_dtypes)


Suggested Data Types for SQL Server:
{'EXT_SEC_ID': 'VARCHAR(MAX)', 'SEC_NAME': 'VARCHAR(MAX)', 'SEC_TYP_CD': 'VARCHAR(MAX)', 'CUSIP': 'VARCHAR(MAX)', 'ISSUER_CD': 'VARCHAR(MAX)', 'ISSUE_STATE_CD': 'VARCHAR(MAX)', 'ISSUE_CNTRY_CD': 'VARCHAR(MAX)', 'ISSUE_DATE': 'VARCHAR(MAX)', 'ULTMT_ISSUER_CD': 'VARCHAR(MAX)', 'LIST_EXCH_CD': 'VARCHAR(MAX)', 'LOC_CRRNCY_CD': 'VARCHAR(MAX)', 'CROS_CRRNCY_CD': 'VARCHAR(MAX)', 'ASSET_CRRNCY_CD': 'VARCHAR(MAX)', 'TICKER': 'VARCHAR(MAX)', 'AVG_LIFE': 'VARCHAR(MAX)', 'MATURE_DATE': 'VARCHAR(MAX)', 'CALL_DATE': 'VARCHAR(MAX)', 'PUT_DATE': 'VARCHAR(MAX)', 'COUPON_RATE': 'VARCHAR(MAX)', 'NEXT_COUPON_DATE': 'VARCHAR(MAX)', 'STRIKE_PRICE': 'VARCHAR(MAX)', 'DURATION': 'VARCHAR(MAX)', 'EXPIRE_DATE': 'VARCHAR(MAX)', 'ALTNTV_MINM_TAX': 'VARCHAR(MAX)', 'RESTR_RESALE': 'VARCHAR(MAX)', 'FED_TXBL': 'VARCHAR(MAX)', 'PRIV_PLCMNT': 'VARCHAR(MAX)', 'VRDN_FREQ_CD': 'VARCHAR(MAX)', 'RESTR_144A': 'VARCHAR(MAX)', 'NON_MKTBL': 'VARCHAR(MAX)', 'NEW_SEC_CD': 'VARCHAR(MAX)', 'MKT_P

# 3. Dropping NULL Columns

In [3]:
import pandas as pd

columns_to_keep = []

for column in df.columns:
    non_null_values = df[column].dropna().tolist()
    if non_null_values:  
        columns_to_keep.append(column)
        print(f"{column}: {non_null_values}")

df_cleaned = df[columns_to_keep]


EXT_SEC_ID: ['abcaa1202121', 'abcaa2012121', 'abcaa240615n', 'abcaa240615o', 'abcaa240815a', 'abcaa240815b', 'abcaa240901g', 'abcaa2409121', 'abcaa2409122', 'abcaa240915i', 'abcaa241001s', 'abcaa241015u', 'abcaa2411121', 'abcaa241115x', 'abcaa2412011', 'abcaa2412208', 'abcaa2501012', 'abcaa2502012', 'abcaa2502121', 'abcaa250215u', 'abcaa2503124', 'abcaa2504062', 'abcaa2504215', 'abcaa2506032', 'abcaa2506091', 'abcaa250615b1', 'abcaa250815a', 'abcaa250815u', 'abcaa251015a4', 'abcaa2511011', 'abcaa2511172', 'abcaa2511206', 'abcaa2512311', 'abcaa2512312', 'abcaa2603011', 'abcaa260415x', 'abcaa2605311', 'abcaa2605313', 'abcaa2605314', 'abcaa260715z', 'abcaa260815o', 'abcaa260815s', 'abcaa2609123', 'abcaa260915n', 'abcaa2610191', 'abcaa2611121', 'abcaa2611162', 'abcaa261215h', 'abcaa270215r', 'abcaa2703011', 'abcaa270415u', 'abcaa270501a', 'abcaa270515m', 'abcaa270615r', 'abcaa2707182', 'abcaa2710111', 'abcaa2711301', 'abcaa2803013', 'abcaa2804011', 'abcaa280615j', 'abcaa2807011', 'abcaa281

In [4]:
df_cleaned

,EXT_SEC_ID,RESTR_144A,CNVRT_RATIO,PAYMENT_FREQ,ANNUAL_DIV
0,abcaa1202121,N,None,None,NaN
1,abcaa2012121,N,None,None,NaN
2,abcaa240615n,N,None,None,NaN
3,abcaa240615o,N,None,None,NaN
4,abcaa240815a,N,None,None,NaN
...,...,...,...,...,...
69826,xscaspswap9,None,None,None,0.0
69827,xsusglobalswap7,None,None,None,0.0
69828,xsusglobalswap9,None,None,None,0.0
69829,xsusmsciacwiswap,None,None,None,0.0


# 4. Finding Duplicates and removing them

In [10]:
import pandas as pd
if 'EXT_SEC_ID' in df_cleaned.columns:
    print("\nAnalyzing 'EXT_SEC_ID' column for duplicates...")

    duplicate_mask = df_cleaned['EXT_SEC_ID'].duplicated(keep=False)
    duplicate_count = df_cleaned['EXT_SEC_ID'].duplicated().sum()
    print(f"Number of duplicate entries in 'EXT_SEC_ID': {duplicate_count}")

    if duplicate_count > 0:
        duplicate_values = df.loc[duplicate_mask, 'EXT_SEC_ID'].drop_duplicates().reset_index(drop=True)
        print("\nDuplicate values in 'EXT_SEC_ID':")
        print(duplicate_values)

        output_file = r"C:\Users\Neha\Desktop\A2_Output.xlsx"
        with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
            duplicate_values.to_excel(writer, sheet_name='Duplicates', index=False, header=['Duplicate ACCT_CD'])

            # Access the workbook and worksheet objects
            workbook = writer.book
            worksheet_duplicates = writer.sheets['Duplicates']

            # Define a format to highlight duplicates in red
            red_format = workbook.add_format({'font_color': 'red', 'bg_color': '#FFC7CE'})

            # Apply red formatting to the 'ACCT_CD' column in the duplicates sheet
            worksheet_duplicates.conditional_format(
                f'A2:A{len(duplicate_values) + 1}',  # Highlight the duplicate ACCT_CD column
                {'type': 'formula', 'criteria': 'TRUE', 'format': red_format}
            )

        print(f"\nExcel file saved with duplicate values at {output_file}")
    else:
        print("\nNo duplicates found in 'EXT_SEC_ID'.")
else:
    print("\n'EXT_SEC_ID' column not found in the DataFrame.")



Analyzing 'EXT_SEC_ID' column for duplicates...
Number of duplicate entries in 'EXT_SEC_ID': 0

No duplicates found in 'EXT_SEC_ID'.


# 5. Changing relevant datatypes in SQL

In [11]:
import pandas as pd
def suggest_datatype(series):
    if pd.api.types.is_integer_dtype(series):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(series):
        return 'FLOAT'
    elif pd.api.types.is_datetime64_any_dtype(series):
        return 'DATETIME'
    elif pd.api.types.is_string_dtype(series):
        return 'VARCHAR(MAX)'
    else:
        return 'VARCHAR(MAX)'
filtered_columns = df_cleaned.columns.tolist()

data_types = {col: suggest_datatype(df_cleaned[col]) for col in filtered_columns}

print(data_types)


{'EXT_SEC_ID': 'VARCHAR(MAX)', 'RESTR_144A': 'VARCHAR(MAX)', 'CNVRT_RATIO': 'VARCHAR(MAX)', 'PAYMENT_FREQ': 'VARCHAR(MAX)', 'ANNUAL_DIV': 'FLOAT'}


# 6. Creating Index

In [8]:
import pyodbc
import pandas as pd

server = 'DELL\\SQLEXPRESS'
database = 'NEHADB'

connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
conn = pyodbc.connect(connection_string)

cursor = conn.cursor()

sql_get_index_column = """
SELECT c.name AS column_name
FROM sys.index_columns ic
JOIN sys.columns c ON ic.column_id = c.column_id AND ic.object_id = c.object_id
JOIN sys.indexes i ON ic.object_id = i.object_id AND ic.index_id = i.index_id
WHERE i.is_primary_key = 1 AND i.object_id = OBJECT_ID('dbo.A2_APX');
"""

cursor.execute(sql_get_index_column)
index_column = cursor.fetchone()

if index_column:
    index_column = index_column[0]  # Fetch the column name

    # Dynamically create an index if it does not exist
    sql_create_index = f"""
    IF NOT EXISTS (
        SELECT 1 
        FROM sys.indexes 
        WHERE object_id = OBJECT_ID('dbo.A2_APX') 
        AND name = 'Index_AF{index_column}'
    )
    BEGIN
        CREATE INDEX Index_AF{index_column}
        ON dbo.A2_APX({index_column});
    END
    """

    cursor.execute(sql_create_index)
    conn.commit()

    # Select the first 5 records
    sql_select_head = "SELECT TOP 5 * FROM dbo.A2_APX"
    data = pd.read_sql(sql_select_head, conn)

    # Reset index and rename the default index column
    data = data.reset_index().rename(columns={'index': 'Index'})
    print(data.head())
else:
    print("No primary key or unique index found for the table dbo.A2_APX")

cursor.close()
conn.close()


   Index    EXT_SEC_ID SEC_NAME SEC_TYP_CD CUSIP ISSUER_CD ISSUE_STATE_CD  \
0      0  abcaa1202121     None       None  None      None           None   
1      1  abcaa2012121     None       None  None      None           None   
2      2  abcaa240615n     None       None  None      None           None   
3      3  abcaa240615o     None       None  None      None           None   
4      4  abcaa240815a     None       None  None      None           None   

  ISSUE_CNTRY_CD ISSUE_DATE ULTMT_ISSUER_CD  ... CREATE_SERVICE_DETAIL  \
0           None       None            None  ...                  None   
1           None       None            None  ...                  None   
2           None       None            None  ...                  None   
3           None       None            None  ...                  None   
4           None       None            None  ...                  None   

  CREATE_SERVICE_CD CREATE_DATE SERVICE_ID SEC_ALT_NAME MEAN_ADJUST_IND  \
0              No

C:\Users\Neha\AppData\Local\Temp\ipykernel_28172\3232985949.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(sql_select_head, conn)
